![background matting](https://i.imgur.com/AtVo6zF.png)

The notebook is split into three parts: required setup, running the algorithm on photos, and running it on videos. You may want to use playground mode to make minor changes, should you need to.

# Required setup

Instructions: Go to Runtime -> Change runtime type and select "GPU" before proceeding. Then, begin by cloning the repo:

In [ ]:
!git clone https://github.com/ydmeira/Background-Matting.git
%cd Background-Matting/

Set up the environment:

In [ ]:
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip3 install tensorflow-gpu==1.14.0
!pip install -r requirements.txt

!git clone https://github.com/tensorflow/models.git
%cd models/research/
import os
os.environ["PYTHONPATH"] += f":{os.getcwd()}:{os.getcwd()}/slim"
%cd ../..

Download model weights for real-fixed-cam and real-hand-held


In [ ]:
!wget https://gist.githubusercontent.com/andreyryabtsev/458f7450c630952d1e75e195f94845a0/raw/0b4336ac2a2140ac2313f9966316467e8cd3002a/download.sh
!chmod +x download.sh
!./download.sh

# Running on images

In [ ]:
#Uncomment and run this if you want to skip uploading your images (below) and run on sample_data instead

cp -r sample_data colab_inputs

## Upload images

Upload your *foreground images* (similar to https://github.com/senguptaumd/Background-Matting/tree/master/sample_data/input) after running the cell below - they will be saved to `colab_inputs` for processing in further cells. Each image you want matted should be uploaded as `NAME_img.png` and `NAME_back.png` for the respective background. Rerun the cell multiple times if you missed some the first time around.

In [ ]:
!mkdir -p colab_inputs/input
%cd colab_inputs/input
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))
%cd ../..

Upload your *target background images* (similar to https://github.com/senguptaumd/Background-Matting/tree/master/sample_data/background) after running the cell below. You can choose the target background to use by name later.


In [ ]:
!mkdir -p colab_inputs/background
%cd colab_inputs/background
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))
%cd ../..

##Process images

Run segmentation


In [ ]:
!python3 test_segmentation_deeplab.py -i colab_inputs/input


Pre-process (transform-align) with AKAZE features. Skip the following cell if your data was captured with a tripod.

In [ ]:
!python test_pre_process.py -i colab_inputs/input

Run matting - replace "0001.png" with the uploaded background you wish to use. Replace "real-hand-held" with "real-fixed-cam" for tripod images for best results.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python test_background-matting_image.py -m real-hand-held -i colab_inputs/input/ -o colab_inputs/output/ -tb colab_inputs/background/0001.png

Visualize results!

In [ ]:
import matplotlib.image as mpimg 
from matplotlib.pyplot import imshow
from glob import glob
%matplotlib inline
results = glob("colab_inputs/output/*_compose.png")
print(results)
if len(results) > 0:
  print(f"Showing {results[0]}")
  testim = mpimg.imread(results[0])
  imshow(testim)


#Running on video

Warning: going through the process on videos, especially in excess of 10 seconds, can take quite some time on Colab

In [ ]:
#Uncomment and run this if you want to skip uploading your videos (below) and run on the sample videos instead (choose fixed or handheld)

!wget https://gist.githubusercontent.com/andreyryabtsev/79d1166452ebd5d5775fbf9169431967/raw/4d20c260d3b3379d1ea9e2e4baab013caf25bc4d/download_vids.sh
!chmod +x download_vids.sh
!./download_vids.sh handheld
# # !./download_vids.sh fixed

## Upload videos


Upload the video from which you would like to extract the foreground, a still of the background, as well as a target background video. Name them `video.mp4`, `video_back.png`, and `target_background.mp4` respectively -- or change the first cell under "process videos" to use your names. You can use any other method to upload if this one takes long (e.g. mount Drive and copy files over).

In [ ]:
!mkdir -p colab_inputs_video
%cd colab_inputs_video
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))
%cd ..

## Process videos

Select the files to use:

In [ ]:
os.environ["BM_SOURCE_VIDEO"] = "video.mp4"
os.environ["BM_SOURCE_STILL"] = "video_back.png"
os.environ["BM_TARGET_VIDEO"] = "target_background.mp4"
os.environ["FRAME_RATE"] = "30" # used only in combining output frames

os.environ["BM_OUTPUT_COMPOSE"] = "video_result.mp4"
os.environ["BM_OUTPUT_MATTE"] = "video_result_matte.mp4"

Extract frames

In [ ]:
%cd colab_inputs_video
%mkdir input background
!ffmpeg -i $BM_SOURCE_VIDEO input/%04d_img.png -hide_banner
!ffmpeg -i $BM_TARGET_VIDEO background/%04d.png -hide_banner
%cd ..

Run segmentation

In [ ]:
!python test_segmentation_deeplab.py -i colab_inputs_video/input

Align the frames -- skip this step if your video was captured with a fixed camera

In [ ]:
!python test_pre_process_video.py -i colab_inputs_video/input -v_name colab_inputs_video/$BM_SOURCE_STILL

Run the matting algorithm. If your video was captured with a fixed camera, comment the first line and uncomment the second before running.

In [ ]:
!git pull

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python test_background-matting_image.py -m real-hand-held -i colab_inputs_video/input/ -o colab_inputs_video/output/ -tb colab_inputs_video/background/
# !CUDA_VISIBLE_DEVICES=0 python test_background-matting_image.py -m real-fixed-cam -i colab_inputs_video/input/ -o colab_inputs_video/output/ -tb colab_inputs_video/background/ -b colab_inputs_video/$BM_SOURCE_STILL

Combine frames into video

In [ ]:
%cd colab_inputs_video
!ffmpeg -r $FRAME_RATE -f image2 -i output/%04d_matte.png -vcodec libx264 -crf 15 -s 1080x1920 -pix_fmt yuv420p $BM_OUTPUT_MATTE
!ffmpeg -r $FRAME_RATE -f image2 -i output/%04d_compose.png -vcodec libx264 -crf 15 -s 1080x1920 -pix_fmt yuv420p $BM_OUTPUT_COMPOSE
%cd ..

The resulting videos can be downloaded from the files tab on the left!